In [3]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

def zero_pad(X, pad):
    
    X_pad = np.pad(X,((0,0), (pad, pad), (pad, pad), (0,0)), 'constant')
    
    return X_pad

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
np.random.seed(1)
x = np.random.rand(4, 3, 3, 2)
x_pad = zero_pad(x, 2)
print(x.shape)
print(x_pad.shape)
print(x[1,1])
print(x_pad[1,1])


(4, 3, 3, 2)
(4, 7, 7, 2)
[[0.87638915 0.89460666]
 [0.08504421 0.03905478]
 [0.16983042 0.8781425 ]]
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [5]:
def conv_single_step(a_slice_prev, W, b):

    s = a_slice_prev * W
    Z = np.sum(s)
    Z = Z + b
    
    return Z

In [6]:
np.random.seed(1)
a_slice_prev = np.random.rand(4, 4, 3)
W = np.random.randn(4,4,3)
b = np.random.randn(1,1,1)

Z = conv_single_step(a_slice_prev, W, b)
print(Z)

[[[2.57809319]]]


In [10]:
def conv_forward(A_prev, W, b, hparameters):
    
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    (f, f, n_C_prev, n_C) = W.shape
    
    stride = hparameters['stride']
    pad = hparameters['pad']
    
    n_H = int((n_H_prev - f + 2*pad) / stride + 1)
    n_W = int((n_W_prev - f + 2*pad) / stride + 1)
    
    Z = np.zeros((m, n_H, n_W, n_C))
    
    A_prev_pad = zero_pad(A_prev, pad)
    
    for i in range(m):
        a_prev_pad = A_prev_pad[i, :, :, :]
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    
                    vert_start = stride * h
                    vert_end = vert_start + f
                    horize_start = stride * w
                    horize_end = horize_start + f
                    
                    a_slice_prev = a_prev_pad[vert_start:vert_end, horize_start:horize_end, :]
                    
                    Z[i, h, w, c] = conv_single_step(a_slice_prev, W[:, :, :, c], b[:, :, :, c])
                    
                    assert(Z.shape) == (m, n_H, n_W, n_C)
                    
                    cache = (A_prev, W, b, hparameters)
                    
                    return Z, cache
            
    

In [12]:
np.random.seed(1)
A_prev = np.random.rand(10, 4, 4, 3)
W = np.random.randn(2, 2, 3, 8)
b = np.random.randn(1,1,1,8)
hparameters = {'pad':2,
               'stride':2}

z, cache = conv_forward(A_prev, W, b, hparameters)
print(np.mean(Z))
print(z[3,2,1])
print(cache[0][1][2][3])

2.578093188905089
[0. 0. 0. 0. 0. 0. 0. 0.]
[0.62367221 0.75094243 0.34889834]
